In [ ]:
!pip install python-terrier
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.0/318.0 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ..

In [ ]:
import pyterrier as pt
from tqdm import tqdm
from pyterrier_t5 import MonoT5ReRanker
import pandas as pd
import gzip
import json
import os
if not pt.started():
  pt.init()
monoT5 = MonoT5ReRanker()

terrier-assemblies 5.7 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.7 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:217: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def rerank_dataset(dataset_name, query_type):
  src_file = f'/content/drive/MyDrive/{dataset_name}.jsonl.gz'
  target_file = f'/content/drive/MyDrive/{dataset_name}-{query_type}-with-scores.jsonl.gz'

  try:
    pd.read_json(target_file, lines=True)
    return
  except:
    pass

  df = []
  with gzip.open(src_file, 'rt') as f:
    for l in tqdm(f, f'Load {src_file}.'):
      l = json.loads(l)
      if l['qid'].endswith(f'_{query_type}'):
        df += [l]
  df = pd.DataFrame(df)

  run = monoT5.transform(df)
  del run['query']
  del run['text']

  run.to_json(target_file, lines=True, orient='records')

In [ ]:
for dataset in ['tot-test-rerank', 'tot-train-rerank', 'tot-dev-rerank']:
    for method in ['orig', '2', '4']:#, '1', ]: #'2']:
        rerank_dataset(dataset, method)

Load /content/drive/MyDrive/tot-dev-rerank.jsonl.gz.: 750000it [00:34, 21685.24it/s]


monoT5:   0%|          | 0/37500 [00:00<?, ?batches/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (651 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
echo "hello"

SyntaxError: ignored

In [ ]:
!zcat /content/drive/MyDrive/tot-test-rerank-tiny-1-with-scores.jsonl.gz|head -50

{"qid":"105_1","docno":"21086574","score":-8.5626335144,"rank":0}
{"qid":"108_1","docno":"42234896","score":-0.5978289247,"rank":0}
{"qid":"118_1","docno":"10904127","score":-1.3696359396,"rank":0}
{"qid":"120_1","docno":"28444122","score":-0.5037651062,"rank":0}
{"qid":"126_1","docno":"35629358","score":-5.7667970657,"rank":0}
{"qid":"132_1","docno":"41446087","score":-3.9461996555,"rank":0}
{"qid":"137_1","docno":"30879026","score":-0.5991196632,"rank":0}
{"qid":"148_1","docno":"876592","score":-0.8530191183,"rank":0}
{"qid":"150_1","docno":"39629878","score":-12.0687179565,"rank":0}
{"qid":"162_1","docno":"60339407","score":-1.2591754198,"rank":0}
{"qid":"171_1","docno":"56339360","score":-9.128077507,"rank":0}
{"qid":"172_1","docno":"2006632","score":-6.2636799812,"rank":0}
{"qid":"173_1","docno":"1336774","score":-0.2414806336,"rank":0}
{"qid":"190_1","docno":"5590094","score":-0.0567402318,"rank":0}
{"qid":"192_1","docno":"35882314","score":-2.2384753227,"rank":0}
{"qid":"200_1",

In [ ]:
!zcat /content/drive/MyDrive/tot-test-rerank-tiny-orig-with-scores.jsonl.gz|head -50

{"qid":"105_orig","docno":"21086574","score":-2.9720499516,"rank":0}
{"qid":"108_orig","docno":"42234896","score":-0.0371814966,"rank":0}
{"qid":"118_orig","docno":"10904127","score":-0.264202565,"rank":0}
{"qid":"120_orig","docno":"28444122","score":-0.0403686687,"rank":0}
{"qid":"126_orig","docno":"35629358","score":-3.9040465355,"rank":0}
{"qid":"132_orig","docno":"41446087","score":-0.0655009076,"rank":0}
{"qid":"137_orig","docno":"30879026","score":-0.6734743118,"rank":0}
{"qid":"148_orig","docno":"876592","score":-0.0547486916,"rank":0}
{"qid":"150_orig","docno":"39629878","score":-0.1446323395,"rank":0}
{"qid":"162_orig","docno":"60339407","score":-0.0135343336,"rank":0}
{"qid":"171_orig","docno":"56339360","score":-0.7459945679,"rank":0}
{"qid":"172_orig","docno":"2006632","score":-0.0406444408,"rank":0}
{"qid":"173_orig","docno":"1336774","score":-0.3372268677,"rank":0}
{"qid":"190_orig","docno":"5590094","score":-0.0303239673,"rank":0}
{"qid":"192_orig","docno":"35882314","sc